In [11]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Fraud Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Fraud Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path
import yaml

@dataclass(frozen=True)
class DataIngestionConfig:
    train_path : Path
    source_train_path: Path
    source_test_path: Path
    test_path : Path
    root_dir: Path

In [6]:
import sys
import os

sys.path.append(os.path.abspath("src"))
from fraud_detection.utils.common import read_yaml, create_directories
from fraud_detection.constants import *

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH):

        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir = config.root_dir,
        source_train_path = config.source_train_path,
        source_test_path = config.source_test_path,
        train_path = config.train_path,
        test_path = config.test_path
        )

        return data_ingestion_config  

In [8]:
import pandas as pd

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config =config
        self.df_train = None
        self.df_test = None

    

    def read_data(self):
        self.df_train = pd.read_csv(self.config.source_train_path,low_memory=False,parse_dates=['Transaction Date'])
        self.df_test = pd.read_csv(self.config.source_test_path,low_memory=False,parse_dates=['Transaction Date'])

    def convert_data_types(self, df:pd.DataFrame) -> pd.DataFrame:
        df['Transaction Amount'] = pd.to_numeric(df['Transaction Amount'], errors='coerce')
        df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
        df['Customer Age'] = pd.to_numeric(df['Customer Age'], errors='coerce')
        df['Account Age Days'] = pd.to_numeric(df['Account Age Days'], errors='coerce')
        df['Transaction Hour'] = pd.to_numeric(df['Transaction Hour'], errors='coerce')
        df['Is Fraudulent'] = df['Is Fraudulent'].astype(int)
        df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
        return df
    
    # def add_date_features(self, df: pd.DataFrame) -> pd.DataFrame:
    #     df['Day of Week'] = df['Transaction Date'].dt.dayofweek
    #     df['Is Weekend'] = df['Day of Week'].isin([5, 6]).astype(int)
    #     return df
    
    def process_and_save(self):

        self.read_data()

        self.df_train = self.convert_data_types(self.df_train)
        #self.df_train = self.add_date_features(self.df_train)

        self.df_test = self.convert_data_types(self.df_test)
        #self.df_test = self.add_date_features(self.df_test)

        self.df_train.to_csv(self.config.train_path,index=False)
        self.df_test.to_csv(self.config.test_path, index=False)

    
    


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    #data_ingestion.read_data()
    data_ingestion.process_and_save()
except Exception as e:
    raise e

[2025-06-30 23:34:20,054: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-30 23:34:20,056: INFO: common: created directory at: artifacts]
[2025-06-30 23:34:20,058: INFO: common: created directory at: artifacts/data_ingestion]
